In [9]:
# make local files accessible to import statements
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../..'))

In [10]:
from examples.Arabas_et_al_2015_Fig_8.demo_controller import DemoController
from examples.Arabas_et_al_2015_Fig_8.storage import Storage
from examples.Arabas_et_al_2015_Fig_8.demo_setup import DemoSetup
from examples.Arabas_et_al_2015_Fig_8.example import Simulation
from examples.Arabas_et_al_2015_Fig_8.demo_viewer import DemoViewer

In [11]:
from IPython.display import display
from ipywidgets import Tab, VBox

In [12]:
setup = DemoSetup()
storage = Storage()
simulation = Simulation(setup, storage)
viewer = DemoViewer(storage, setup)
controller = DemoController(simulation, viewer)
tabs = Tab([VBox([controller.box(), viewer.box()]), setup.box()])
tabs.set_title(1, "Setup")
tabs.set_title(0, "Simulation")
tabs.observe(controller.reinit, 'selected_index')
display(tabs)